In [ ]:
# pip install -r requirements.txt

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#from autoaugment import ImageNetPolicy

In [ ]:
# Use GPU if it's available
if torch.cuda.is_available():
    device = 'cuda'
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    device = 'cpu'
    torch.set_default_tensor_type(torch.FloatTensor)

# nel mio caso non dispongo di abbastanza RAM nella GPU per un'elaborazione su scheda grafica
# device = torch.device("cpu")

In [ ]:
orig_file = r'/home/davide/Documents/Progetto/dataset/FoodSplitted'
grey_hist = r'/home/davide/Documents/Progetto/dataset/grey_hist_coversion'

data_dir = grey_hist

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [ ]:
data_dir = r'/home/davide/Documents/Progetto/dataset/grey_hist_coversion'

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + r'/train', transform=transform)
val_data = datasets.ImageFolder(data_dir + r'/val', transform=transform)
test_data=datasets.ImageFolder(data_dir + r'/test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=6)
test_loader=torch.utils.data.DataLoader(test_data, batch_size=128)

In [ ]:

# # TODO: Define transforms for the training data and testing data
# train_transforms = transforms.Compose([transforms.RandomRotation(30),
#                                        transforms.RandomResizedCrop(224),
#                                        transforms.RandomHorizontalFlip(),
#                                        #ImageNetPolicy(),
#                                        transforms.ToTensor(),
#                                        transforms.Normalize([0.485, 0.456, 0.406],
#                                                             [0.229, 0.224, 0.225])])

# test_transforms = transforms.Compose([transforms.Resize(255),
#                                       transforms.CenterCrop(224),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406],
#                                                            [0.229, 0.224, 0.225])])

# # Pass transforms in here, then run the next cell to see how the transforms look
# train_data = datasets.ImageFolder(data_dir + r'/train', transform=train_transforms)
# val_data = datasets.ImageFolder(data_dir + r'/val', transform=test_transforms)
# test_data=datasets.ImageFolder(data_dir + r'/test', transform=test_transforms)

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
# val_loader = torch.utils.data.DataLoader(val_data, batch_size=128)
# test_loader=torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
model =models.densenet121(pretrained=True)
model

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 101)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [ ]:
criterion = nn.CrossEntropyLoss()

adam_optimizer = optim.Adam(model.classifier.parameters(), lr=0.001, betas=[0.9, 0.999])

RMSProp_optimizer = optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

In [ ]:
import numpy as np
import time
def train(n_epochs,trainloader,testloader, model, optimizer, criterion, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
  
    for epoch in range(n_epochs):
        
        start = time.time()
        print('\n\t-----------------\n')
        print(start,'\n')
        
        running_loss=0
        model.train() 
        
        for inputs, labels in trainloader:
        
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)
            # inputs, labels = inputs.cuda(), labels.cuda()
            
            # we need to set the gradients to zero before starting to do backpropragation
            # because PyTorch accumulates the gradients on subsequent backward passes.
            optimizer.zero_grad()
        
            logps = model(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
        
        print('running_loss: ', running_loss)
        print(f"time trainloader: {(time.time() - start):.3f} seconds\n")    
        
        model.eval()
        valid_loss=0
        accuracy=0
        with torch.no_grad():
            # Disabling gradient calculation is useful for inference,
            # when you are sure that you will not call Tensor.backward(). 
            # It will reduce memory consumption for computations 
            # that would otherwise have requires_grad=True.
            
            for inputs, labels in testloader:
                # inputs, labels = inputs.cuda(), labels.cuda()
                inputs, labels = inputs.to(device), labels.to(device)
                
                logps = model(inputs)
                batch_loss = criterion(logps, labels)
                valid_loss += batch_loss.item()
                    
                # Calculate accuracy
                
                top_p, top_class = logps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                
        
            if valid_loss < valid_loss_min:
                print("Validation loss decreased. Saving model")
                torch.save(model.state_dict(),save_path)
                valid_loss_min=valid_loss
                
                    
            
            print(f"Device = {device} ; Time per batch: {(time.time() - start):.3f} seconds")       
            print(f"Epoch: {epoch+1}/{n_epochs}.. "
                  f"Train loss: {running_loss/len(trainloader):.3f}.. "
                  f"Test loss: {valid_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}\n")       

In [ ]:
# model.load_state_dict(torch.load('denseNet_food101.pt'))

In [ ]:
train(5,train_loader,val_loader, model, RMSProp_optimizer, criterion,'denseNet_food101_5_220621_RMSProp.pt')
# train(5,trainloader,testloader, model, sgd_optimizer, criterion,'denseNet_food101_5_sgd.pt')

In [ ]:
# torch.save(model.state_dict(),'denseNet_food101_10_noTran_220621_adam.pth')

In [ ]:
for param in model.parameters():
    param.requires_grad = False

print('Training these layers')
for name,param in model.named_parameters():
    print(name, param.requires_grad)
    if name.startswith('features.denseblock4.denselayer16'):
        param.requires_grad = True
#         print('---->', name, param.requires_grad)

In [ ]:
# model.load_state_dict(torch.load('denseNet_food101_5_noTran_220621_RMSProp.pt'))

In [ ]:
train(5,train_loader,val_loader, model, adam_optimizer, criterion,'denseNet_food101_5_rmsporp_5_adam_220621.pt')

In [ ]:
torch.save(model.state_dict(),'denseNet_food101_5_rmsprop_5_adam.pth')

In [ ]:
# model.load_state_dict(torch.load('denseNet_food101.pth'))

In [ ]:
valid_loss=0
accuracy=0
with torch.no_grad():
  model.eval()
  for images,labels in test_loader:
    # images,lables=images.cuda(),labels.cuda()
    images,lables=images.to(device),labels.to(device)
    logps = model(images)
    batch_loss = criterion(logps, labels)
    valid_loss += batch_loss.item()
    top_p, top_class = logps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
print(valid_loss/len(test_loader))
print(accuracy/len(test_loader))